# Operations Research 3: Examples for Theory

This code is for students who take the class Operations Research. Students should finish the installation of Gurobi and Python before workersed and make sure an academic liscense for Gurobi is applied and activated.

We introduce two examples for shadow price in order to help students understand how to implement theories introduced in lectures with codes. More instruction is provided in the lecture video.

# Shadow Price
## Example 1: Producing desks and tables


Consider the problem we introduced in Operations Research: Modeling and Application, we have

\begin{split}
    \begin{array}{r}
        \max \\ \mbox{s.t.} \\ \\ \\ \\ \\ 
    \end{array} &
    \begin{array}{rcrcll}
        700x_1 & + & 900x_2 & & & \\ 
        3x_1 & + & 5x_2 & \leq & 3600\quad\! & \mbox{(wood)} \\		
        x_1 & + & 2x_2 & \leq & 1600\quad\! & \mbox{(labor)} \\		
        50x_1 & + & 20x_2 & \leq & 48000\:\ & \mbox{(machine)} \\
        x_1 & & & \geq & 0
        \\
        & & x_2 & \geq & 0.
    \end{array}
\end{split}


We use the code introduced in Operations Research: Algorithms to show shadow prices Gurobi.\
Let's construct our problem step by step.

In [1]:
from gurobipy import *

In [2]:
# change the color and font style when printing string

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

Different from the code we used before, we put our data and model in a fuction to make adjust the resource limitation (the RHS of constraint) more easily.

In [3]:
def linear_example(limitation_list):

    """Data Part"""

    products = range(2)  # 2 products    
    resources = range(3)  # 3 resources

    prices = [700, 900]    
    resource_consumptions = [[3 , 5 ],
                             [1 , 2 ],
                             [50, 20]]
    resource_limitations = limitation_list


    """Model Part"""

    eg1 = Model("LP_example")

    x = []
    for i in products:
        x.append(eg1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'x' + str(i)))

    eg1.setObjective(quicksum(prices[i] * x[i] for i in products) , GRB.MAXIMIZE) 

    # add constraints and name them
    eg1.addConstrs((quicksum(resource_consumptions[j][i] * x[i] for i in products) 
                               <= resource_limitations[j] for j in resources), "Resource_limitation")

    eg1.optimize()

    for var in eg1.getVars():
        print(var.varName, '=', round(var.x, 2))
    print("objective value =", round(eg1.objVal, 2))
    
    return eg1

Call the original model and set the original resource limitation set.

In [4]:
limitation_list = [3600, 1600, 48000]
origin_model1 = linear_example(limitation_list)

# use Pi to get the shadow price of each constraint
print('The shadow price of the wood constraint is {shadow_price}.'.format(shadow_price = origin_model1.Pi[0]))
print('The shadow price of the labor constraint is {shadow_price}.'.format(shadow_price = origin_model1.Pi[1]))
print('The shadow price of the machine constraint is {shadow_price}.'.format(shadow_price = origin_model1.Pi[2]))

Using license file /Users/jennytzeng/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xa395d65c
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [7e+02, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 5e+04]
Presolve time: 0.01s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+32   3.593750e+30   2.000000e+02      0s
       3    7.8947368e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds
Optimal objective  7.894736842e+05
x0 = 884.21
x1 = 189.47
objective value = 789473.68
The shadow price of the wood constraint is 163.1578947368421.
The shadow price of the labor constraint is 0.0.
The shadow price of the machine constraint is 4.2105263157894735.


Use a loop to adjust the three constraints in our model.

In [5]:
for i in range(3):
    print('==============================================')
    
    # print the shadow prices of constraints and review the expected objective value
    print(color.DARKCYAN + color.BOLD + 
          'Shadow price of {name} is {shadow_price}'.format(name = origin_model1.ConstrName[i], shadow_price = round(origin_model1.Pi[i], 2)) 
          + color.END)
    
    # print the original objective value
    print(color.DARKCYAN + color.BOLD + 
          'Original objective value: {obj}'.format(obj = round(origin_model1.objVal, 2)) + color.END)
    
    # set new RHS
    new_limitation_list = limitation_list.copy()
    new_limitation_list[i] = limitation_list[i] + 1
    
    # construct the model with new constraints
    new_model1 = linear_example(new_limitation_list)
    print(color.DARKCYAN + color.BOLD + 
          'Objective value after adding {name} by one unit: {obj}'.format(name = new_model1.ConstrName[i],obj = round(new_model1.objVal, 2)) 
          + color.END)

Shadow price of Resource_limitation[0] is 163.16
Original objective value: 789473.68
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0x78fe43bc
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [7e+02, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 5e+04]
Presolve time: 0.01s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+32   3.593750e+30   2.000000e+02      0s
       3    7.8963684e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds
Optimal objective  7.896368421e+05
x0 = 884.11
x1 = 189.74
objective value = 789636.84
Objective value after adding Resource_limitation[0] by one unit: 789636.84
Shadow price of Resource_limitation[1] is 0.0
Original objective value: 789473.68
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model wit

## Example 2: Personnel Scheduling

Consider the personnel scheduling problem we introduced in Operations Research: Modeling and Application, we have

\begin{split}
    \begin{array}{rrllllllllllllll}
        \min \ & x_1 & + & x_2 & + & x_3 & + & x_4 & + & x_5 & + & x_6 & + & x_7  \\
        \mbox{s.t.} \ & x_1 & + &&&&& x_4 & + & x_5 & + & x_6 & + & x_7 & \geq 110\\
        & x_1 & + & x_2 & + &&&&& x_5 & + & x_6 & + & x_7 & \geq 80\\
        & x_1 & + & x_2 & + & x_3 & + &&&&& x_6 & + & x_7 & \geq 150\\
        & x_1 & + & x_2 & + & x_3 & + & x_4 & + &&&&& x_7 & \geq 30\\
        & x_1 & + & x_2 & + & x_3 & + & x_4 & + & x_5 &&&&& \geq 70\\
        &&& x_2 & + & x_3 & + & x_4 & + & x_5 & + & x_6 &&& \geq 160\\
        &&&&& x_3 & + & x_4 & + & x_5 & + & x_6 & + & x_7 & \geq 120\\
    \end{array} \\
    x_i \geq 0 \quad \forall i = 1,...,7.\hspace{9.1cm}
\end{split}

We also used the code intoduced in Operations Research: Algorithms.
Let's construct the problem and check the shadow prices of the model.

In [6]:
from gurobipy import *
import pandas as pd

First, we import personnel scheduling constraints which is limitations of people needed in each day. More details were mentioned in Operations Research: Modeling and Application.

In [7]:
personnel_con = pd.read_excel('personnel_scheduling.xlsx', 'people_needed')
print(personnel_con)

  weekday  people_needed
0     Mon            110
1     Tue             80
2     Wed            150
3     Thu             30
4     Fri             70
5     Sat            160
6     Sun            120


Same as th last example, we put the code of data and model part into a function.

In [8]:
def personnel_scheduling_example(constraints = personnel_con['people_needed']):

    """Data Part"""

    people = range(7)  # 7 numbers of people work in each day of a week
    days = range(7)  # 7 days in a week 

    day_limitations = constraints


    """Model Part"""

    eg2 = Model("personnel_scheduling")
    
    # add variables
    x = []
    for i in people:
        x.append(eg2.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'x' + str(i)))
    
    # set objective value
    eg2.setObjective(quicksum(x[i] for i in people), GRB.MINIMIZE) 

    # add constraints and name them
    eg2.addConstrs((quicksum(x[i + j - 7] for i in range(3, 8)) 
                               >= day_limitations[j] for j in days), "Resource_limitation")

    eg2.optimize()

    for var in eg2.getVars():
        print(var.varName, '=', round(var.x, 2))
    print("objective value =", round(eg2.objVal, 2))
    
    return eg2

In [9]:
origin_model2 = personnel_scheduling_example()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 7 rows, 7 columns and 35 nonzeros
Model fingerprint: 0xa040a86d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 2e+02]
Presolve time: 0.02s
Presolved: 7 rows, 7 columns, 35 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.200000e+02   0.000000e+00      0s
       5    1.6333333e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.05 seconds
Optimal objective  1.633333333e+02
x0 = 3.33
x1 = 40.0
x2 = 13.33
x3 = 13.33
x4 = 0.0
x5 = 93.33
x6 = 0.0
objective value = 163.33


Use loop to view the shadow prices of each constraints. We check the shadow prices and the expected objective value, and then compare the different between expected and actual objective value.

In [10]:
for i in range(7):
    print('==============================================')
    
    # print the shadow prices of constraints and review the expected objective value
    print(color.DARKCYAN + color.BOLD + 
          'Shadow price of {name} is {shadow_price}'.format(name = origin_model2.ConstrName[i], shadow_price = round(origin_model2.Pi[i], 2)) 
          + color.END)
    print(color.DARKCYAN + color.BOLD + 'Original objective value: {obj}'.format(obj = round(origin_model2.objVal, 2)) + color.END)
    
    # set new RHS
    new_days_limitation = personnel_con['people_needed'].copy()
    new_days_limitation[i] = new_days_limitation[i] + 1
    new_model2 = personnel_scheduling_example(new_days_limitation)
    print(color.DARKCYAN + color.BOLD + 
          'Objective value after adding {name} one unit: {obj}'.format(name = new_model2.ConstrName[i],obj = round(new_model2.objVal, 2)) 
          + color.END)

Shadow price of Resource_limitation[0] is 0.33
Original objective value: 163.33
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 7 rows, 7 columns and 35 nonzeros
Model fingerprint: 0xa73572d6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 2e+02]
Presolve time: 0.00s
Presolved: 7 rows, 7 columns, 35 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.210000e+02   0.000000e+00      0s
       5    1.6366667e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds
Optimal objective  1.636666667e+02
x0 = 3.67
x1 = 40.0
x2 = 12.67
x3 = 13.67
x4 = 0.0
x5 = 93.67
x6 = 0.0
objective value = 163.67
Objective value after adding Resource_limitation[0] one unit: 163.67
Shadow price of Resource_limitation[1] is 0.0
Original objective value: 163.33
Gurobi Optimizer version 9.0.2 build v9.0.2